In [4]:
#Modules to install via pip pandas,ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pprint import pprint
import os
import import_ipynb
import sys
import kmeans
sys.path.append('../')
from functions import *
from trace_analysis import *
from plots import *
from trace_analysis_cooja2 import *
from node import *
from plots_analysis import *
from pandas.plotting import scatter_matrix
import cmath as math
from mpl_toolkits.mplot3d import Axes3D
from sklearn import cluster
from sklearn.metrics import confusion_matrix
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.cluster import KMeans
# scipy
from scipy.cluster.vq import kmeans,vq,whiten
import sklearn.metrics as sm
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
%matplotlib inline
import random
random.seed(6666)

In [15]:
def analyze_network(directory,plots,pings,window):
    cases=[]
    casesAccuracy=[]
    for row in plots:
        cases.append(row[1])
        casesAccuracy.append(row[2])
        data=import_Cooja2(plots)
    pings=getPings(data)
    #All data collection is in variable node that is a list of list of nodes
    #3 nets input x 9 nodes by net

    d={ "label":[],
       "type":[],
        "count":[],
        "std":  [],
        "mean": [],
        "var":  [],
        "hop":[],

       "packet loss":[],
       "outliers":[],
       "node":[]
    }
    #count=[]
    labels=[]
    var=[]
    #window=100
    #stats=pd.DataFrame(columns=columns)

    for i in range(len(data)):
        #window=pings[i]

        for j in range(len(data[i])):
            #n=pings[i]
            n=100
            window=50
            #print(n)
            for z in range(0,n,int(window)):
                #if(z+window>n):break
                #print(z,z+window)

                #df1 = df1.assign(e=p.Series(np.random.randn(sLength)).values)
                node=data[i][j].pkts
                name=str(j)+" "+cases[i]
                nodeWindow=node[(node["seq"]<z+window) & (node["seq"]>=z)]
                nodeWindowP=nodeWindow["rtt"]
                d["count"].append(nodeWindowP.count())
                #Case without outliers
                #Case with outliers
                std=0
                if (nodeWindowP.std()>10):
                    std=1
                    std=nodeWindowP.std()

                d["std"].append(std)
                mean=nodeWindowP.mean()
                #if(mean<1):print(mean)
                d["mean"].append(mean)
                var=0
                if (nodeWindowP.var()>var): var=nodeWindowP.var()
                d["var"].append(var)
                d["label"].append(cases[i])
                d["hop"].append(data[i][j].hop)
                d["type"].append(casesAccuracy[i])
                d["outliers"].append(getOutliers(nodeWindow)["rtt"].count())
                missing=window-nodeWindow.count()
                d["node"].append(data[i][j].ip)
                mP=getPercentageMissingPackets(nodeWindow,window)
                PL=0
                if(mP>30):
                    PL=1
                    PL=mP
                d["packet loss"].append(PL)



def analyze_network(directory,plots,pings,window):
    cases=[]
    casesAccuracy=[]
    for row in plots:
        cases.append(row[1])
        casesAccuracy.append(row[2])
        data=import_Cooja2(plots)
    pings=getPings(data)
    #All data collection is in variable node that is a list of list of nodes
    #3 nets input x 9 nodes by net

    d={ "label":[],
       "type":[],
        "count":[],
        "std":  [],
        "mean": [],
        "var":  [],
        "hop":[],

       "packet loss":[],
       "outliers":[],
       "node":[]
    }
    #count=[]
    labels=[]
    var=[]
    #window=100
    #stats=pd.DataFrame(columns=columns)

    for i in range(len(data)):
        #window=pings[i]

        for j in range(len(data[i])):
            #n=pings[i]
            n=100
            window=50
            #print(n)
            for z in range(0,n,int(window)):
                #if(z+window>n):break
                #print(z,z+window)

                #df1 = df1.assign(e=p.Series(np.random.randn(sLength)).values)
                node=data[i][j].pkts
                name=str(j)+" "+cases[i]
                nodeWindow=node[(node["seq"]<z+window) & (node["seq"]>=z)]
                nodeWindowP=nodeWindow["rtt"]
                d["count"].append(nodeWindowP.count())
                #Case without outliers
                #Case with outliers
                std=0
                if (nodeWindowP.std()>10):
                    std=1
                    std=nodeWindowP.std()

                d["std"].append(std)
                mean=nodeWindowP.mean()
                #if(mean<1):print(mean)
                d["mean"].append(mean)
                var=0
                if (nodeWindowP.var()>var): var=nodeWindowP.var()
                d["var"].append(var)
                d["label"].append(cases[i])
                d["hop"].append(data[i][j].hop)
                d["type"].append(casesAccuracy[i])
                d["outliers"].append(getOutliers(nodeWindow)["rtt"].count())
                missing=window-nodeWindow.count()
                d["node"].append(data[i][j].ip)
                mP=getPercentageMissingPackets(nodeWindow,window)
                PL=0
                if(mP>30):
                    PL=1
                    PL=mP
                d["packet loss"].append(PL)



    stats=pd.DataFrame(d)

    dataK=stats.drop([
        "label",
        "mean",
        "var",
        "std",
        #"packet loss",
        "outliers",
        "hop",
        "count",
        "node",
        #"type"
    ],axis=1)
    dataK=dataK.fillna(0)
    correction=[]
    correction_alt=[] #fr 3 cluster 0:normal net 1:bh net 2:bh
    col=np.array(dataK["type"])
    dataK=dataK.drop(["type"],axis=1)
    #Creating simple array to correct unsupervised learning
    #NB as it is unsupervised could happen that the correction are inverted
    for i in range(len(col)):
        el=d["type"][i]
        if el=="normal":
            correction.append(1)
            correction_alt.append(0)

        else:
            #print(el=="BH2" and i==BlackHole[2])
            correction.append(0)
            correction_alt.append(1)


    dataC=stats["label"]

    #Y = data[['var']]

    #X = data[['std']]

    kmeans = KMeans(n_clusters=2)
    kmeans.fit(dataK)
    labels = kmeans.predict(dataK)
    centroids = kmeans.cluster_centers_
    labels=accuracy_score_corrected(correction,labels)
    predicted=[]
    for i in range(len(labels)):

        if(labels[i]==1):
            predicted.append("normal")
        else: predicted.append("BH")

    #print(len(predicted))
    stats["predicted"]=pd.Series(np.array(predicted))
    stats["predicted number"]=pd.Series(np.array(labels))
    stats["correction number"]=pd.Series(np.array(correction))
    stats_csv=stats[[
        "label",
        "type",
        "predicted",
        "packet loss",
        "outliers",
        "std",
        "hop",
        "node",
        "mean"


          ]]
    stats_csv.to_csv("results_kmeans.csv", sep='\t', encoding='utf-8')
    stats.head()
    net_results={
       "case":[],
        "predicted":[],
        "real":[]
    }
    #print(stats["predicted number"])
    for case in range(len(cases)):
        subset=stats[stats["label"]==cases[case]]
        mean_predicted=str(subset["predicted number"].mean()*100)+"% normal"
        net_results["case"].append(cases[case])
        net_results["predicted"].append(mean_predicted)
        net_results["real"].append(casesAccuracy[case])



    results=pd.DataFrame(net_results)
    results.to_csv("results_network_kmeans.csv", sep='\t', encoding='utf-8')
    print(results)


In [14]:
#directory=os.getcwd()+"/cooja3-9nodes/"
directory="../cooja3-9nodes/"
plots = [
        #2 BH3
        (directory+"traces/1bh-3", 'grid9_1bh-3_2019-02-13_16:28_',"BH"),
        (directory+"traces/1bh-3", 'grid9_1bh-3_2019-02-13_22:05_',"BH"),
        #2 BH5
         (directory+"traces/1bh-5", 'grid9_1bh-5_2019-02-13_15:31_',"BH"),
          (directory+"traces/1bh-5", 'grid9_1bh-5_2019-02-13_21:44_',"BH"),
        #2 BH 6
        (directory+"traces/1bh-6", 'grid9_1bh-6_2019-02-13_12:59_',"BH"),
        (directory+"traces/1bh-6", 'grid9_1bh-6_2019-02-13_19:15_',"BH"),
        #2 BH 7
         (directory+"traces/1bh-7", 'grid9_1bh-7_2019-02-13_15:08_',"BH"),
         (directory+"traces/1bh-7", 'grid9_1bh-7_2019-02-13_20:02_',"BH"),
         #2 bh 9
         (directory+"traces/1bh-9", 'grid9_1bh-9_2019-02-13_15:57_',"BH"),
         (directory+"traces/1bh-9", 'grid9_1bh-9_2019-02-13_19:35_',"BH"),
         #3 normal
         (directory+"traces/normal", 'grid9_normal_2019-02-13_17:05_',"normal"),
         (directory+"traces/normal",  "grid9_normal_2019-02-13_18:51_","normal"),
         (directory+"traces/normal",  "grid9_normal_2019-02-13_22:23_","normal"),
        ]

analyze_network(directory,plots,100,100)



[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]
0.47863247863247865
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]
array([[ 58, 122],
       [ 